In [ ]:
# Training Data
!wget https://www.dropbox.com/sh/gmgduvzyl5tken6/AAD6E17hrD8CfKwmRB9wfTKqa/X_reference.npy
!wget https://www.dropbox.com/sh/gmgduvzyl5tken6/AACaXFPq1qA5kLvhIF0Y0NfEa/y_reference.npy

# Testing Data
!wget https://www.dropbox.com/sh/gmgduvzyl5tken6/AABKsseO9QvObI4lTt-yXg8ta/X_test.npy
!wget https://www.dropbox.com/sh/gmgduvzyl5tken6/AACitgFQG5cfd7qs--swRbI0a/y_test.npy

In [45]:
# import packages
from tensorflow.keras.layers import BatchNormalization, ZeroPadding1D, Conv1D, AveragePooling1D, MaxPooling1D
from tensorflow.keras.layers import Activation, Dense,  Flatten, Input, add
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np

#define resnet class
class ResNet:
  @staticmethod
  #create one residual block
  def residual_module(data, K, stride, chanDim=-1, reduce=False, reg=0.0001, bnEps=2e-5, bnMom=0.9):
    shortcut = data
    
    bn1 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(data)
    act1 = Activation("relu")(bn1)
    conv1 = Conv1D(int(K*0.25), 1, use_bias=False, kernel_regularizer=l2(reg))(act1)

    bn2 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(conv1)
    act2 = Activation("relu")(bn2)
    conv2 = Conv1D(int(K * 0.25), 1, strides=stride, use_bias=False, kernel_regularizer=l2(reg))(act2)
    
    bn3 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(conv2)
    act3 = Activation("relu")(bn3)
    conv3 = Conv1D(K, 1, use_bias=False, kernel_regularizer=l2(reg))(act3)

    if reduce:
      shortcut = Conv1D(K, 1, strides=stride, use_bias=False, kernel_regularizer=l2(reg))(act1)

    x = add([conv3, shortcut])

    return x

  @staticmethod
  #build a model out of resblocks
  def build(inputShape, classes, stages, filters, reg=0.0001, bnEps=2e-5, bnMom=0.9):
    chanDim = -1
    inputs = Input(shape=inputShape)

    #initial batchnorm and convolutional layer
    x = BatchNormalization(axis=chanDim, epsilon = bnEps, momentum=bnMom)(inputs)
    x = Conv1D(filters[0], 5, use_bias=False, padding="same", kernel_regularizer=l2(reg))(x)
    
    x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(x)
    x = Activation("relu")(x)
    x = ZeroPadding1D(1)(x)
    x = MaxPooling1D(3, strides=2)(x)

    for i in range(0, len(stages)):
      if i == 0:
        stride = 1
      else:
        stride = 2
      
      x = ResNet.residual_module(data=x, K=filters[i + 1], stride=stride, reduce=True, bnEps=bnEps, bnMom=bnMom)

      for j in range(0, stages[i] - 1):
        x = ResNet.residual_module(data=x, K=filters[i + 1], stride=1, bnEps=bnEps, bnMom=bnMom)
      
      x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(x)
      x = Activation("relu")(x)
      x = AveragePooling1D(8)(x)

      x = Flatten()(x)
      x = Dense(classes, kernel_regularizer=l2(reg))(x)
      x = Activation("softmax")(x)

      model = Model(inputs, x, name="resnet")

      return model

In [49]:
#some hyperparameters
batch_size = 32
epochs = 10
verbose = 2
stages = (6,)
filters = (64, 100)
resLayers = 6
convLayers = 4


#load data (specific to google drive)
'''x_train = np.load('X_reference.npy')
y_train = np.load('y_reference.npy')
x_test = np.load('X_test.npy')
y_test = np.load('y_test.npy')
'''
X = np.load('X_reference.npy')
y = np.load('y_reference.npy')

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20)


#add depth channel
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

#input shape is everything except for the number of samples 
#number of classes is the number of unique items in y
in_shape = x_train.shape[1:]
n_classes = len(np.unique(y_train))

#for debugging
print(x_train.shape, in_shape)

#build model
model = ResNet.build(inputShape=in_shape, classes=n_classes, stages=stages, filters=filters)

#compile, summarize, train model
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()
model.fit(x_train, y_train, batch_size=batch_size,epochs=epochs, verbose=verbose)

#test accuracy
accuracy = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=verbose)
print(accuracy)

#delete model
del model


(48000, 1000, 1) (1000, 1)
Model: "resnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_23 (InputLayer)           [(None, 1000, 1)]    0                                            
__________________________________________________________________________________________________
batch_normalization_185 (BatchN (None, 1000, 1)      4           input_23[0][0]                   
__________________________________________________________________________________________________
conv1d_172 (Conv1D)             (None, 1000, 64)     320         batch_normalization_185[0][0]    
__________________________________________________________________________________________________
batch_normalization_186 (BatchN (None, 1000, 64)     256         conv1d_172[0][0]                 
__________________________________________________________________